In [11]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
from pandas.tseries.holiday import USFederalHolidayCalendar
from sklearn.model_selection import train_test_split
import glob
import random

# Set random seed
seed = 391
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)

# Load adjacency matrix
adj_df = pd.read_csv('WECC_adjacent.csv', index_col=0)
edge_index = torch.tensor(np.array(np.where(adj_df.values == 1)), dtype=torch.long)

ba_names = list(adj_df.index)
ba_files_dict = {file.split('/')[-1].replace('_historical_data.csv', ''): file
                 for file in glob.glob('./WECC_compiled_historical_data/*.csv')}

# Reorder the files to match ba_names
# ba_files_order = [ba_files_dict[ba] for ba in ba_names]

x_train_node_features_list = []
y_train_node_features_list = []
x_val_node_features_list = []
y_val_node_features_list = []
x_test_node_features_list = []
y_test_node_features_list = []

min_y_train_list = []
max_y_train_list = []

dates_list = []
# ba_files = glob.glob('./WECC_compiled_historical_data/*.csv')

for ba_name in ba_names:
    file = ba_files_dict[ba_name]
    df = pd.read_csv(file)

    # Convert to datetime
    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour']])
    
    # Remove rows with missing or infinite values
    features = ['Hour', 'Month', 'T2', 'Q2', 'WSPD', 'GLW', 'SWDOWN']
    target = 'Adjusted_Demand_MWh'
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.dropna(subset=features + [target])
    
    dates_list.append(set(df['Date'].unique()))

common_dates = set.intersection(*dates_list)
common_dates = sorted(list(common_dates))
print(f"Number of common dates: {len(common_dates)}")

for ba_name in ba_names:
    file = ba_files_dict[ba_name]
    df = pd.read_csv(file)

    # Convert to datetime
    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour']])
    df['Date1'] = pd.to_datetime(df[['Year', 'Month', 'Day']])

    # Add weekday feature
    df['Is_Weekday'] = df['Date1'].dt.weekday  # 0=Monday, 6=Sunday
    df['Weekday'] = (df['Is_Weekday'] < 5).astype(int)

    # Add holiday feature
    calendar = USFederalHolidayCalendar()
    holidays = calendar.holidays(start=df['Date1'].min(), end=df['Date1'].max())
    df['Holiday'] = df['Date1'].isin(holidays).astype(int)

    # Features and target
    features = ['Hour', 'Month', 'T2', 'Q2', 'WSPD', 'GLW', 'SWDOWN', 'Weekday', 'Holiday']
    target = 'Adjusted_Demand_MWh'

    # Keep only common dates
    df = df[df['Date'].isin(common_dates)]
    
    # Split by year
    data = df[df['Year'].isin([2016, 2017, 2018])]
    # val_data = df[df['Year'] == 2018]
    test_data = df[df['Year'] == 2019]

    # Prepare X and y
    X = data[features].values
    y = data[target].values.reshape(-1, 1)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=391)

    # X_train = train_data[features].values
    # y_train = train_data[target].values.reshape(-1, 1)
    # X_val = val_data[features].values
    # y_val = val_data[target].values.reshape(-1, 1)
    X_test = test_data[features].values
    y_test = test_data[target].values.reshape(-1, 1)

    # Standardize
    min_x_train = np.min(X_train, axis=0)
    max_x_train = np.max(X_train, axis=0)
    min_y_train = np.min(y_train, axis=0)
    max_y_train = np.max(y_train, axis=0)

    min_y_train_list.append(min_y_train[0])  # save scalar
    max_y_train_list.append(max_y_train[0])

    x_train_norm = np.divide((X_train - min_x_train), (max_x_train - min_x_train))
    x_val_norm = np.divide((X_val - min_x_train), (max_x_train - min_x_train))
    x_test_norm = np.divide((X_test - min_x_train), (max_x_train - min_x_train))
    y_train_norm = np.divide((y_train - min_y_train), (max_y_train - min_y_train)).squeeze()
    y_val_norm = np.divide((y_val - min_y_train), (max_y_train - min_y_train)).squeeze()
    y_test_norm = np.divide((y_test - min_y_train), (max_y_train - min_y_train)).squeeze()

    x_train_node_features_list.append(x_train_norm)  # shape: (N_samples, num_features)
    y_train_node_features_list.append(y_train_norm)
    x_val_node_features_list.append(x_val_norm)  
    y_val_node_features_list.append(y_val_norm)
    x_test_node_features_list.append(x_test_norm)  
    y_test_node_features_list.append(y_test_norm)

# Stack into [samples, nodes, features]
x_np = np.stack(x_train_node_features_list, axis=1)
y_np = np.stack(y_train_node_features_list, axis=1)
x_val_np = np.stack(x_val_node_features_list, axis=1)
y_val_np = np.stack(y_val_node_features_list, axis=1)
x_test_np = np.stack(x_test_node_features_list, axis=1)
y_test_np = np.stack(y_test_node_features_list, axis=1)

min_y_train_array = np.array(min_y_train_list)  # shape: (nodes,)
max_y_train_array = np.array(max_y_train_list)  # shape: (nodes,)

# Tensor preparation
x_tensor = torch.tensor(x_np, dtype=torch.float)
y_tensor = torch.tensor(y_np, dtype=torch.float)
x_val_tensor = torch.tensor(x_val_np, dtype=torch.float)
y_val_tensor = torch.tensor(y_val_np, dtype=torch.float)
x_test_tensor = torch.tensor(x_test_np, dtype=torch.float)
y_test_tensor = torch.tensor(y_test_np, dtype=torch.float)

Number of common dates: 35138
{'TPWR': './WECC_compiled_historical_data/TPWR_historical_data.csv', 'GCPD': './WECC_compiled_historical_data/GCPD_historical_data.csv', 'DOPD': './WECC_compiled_historical_data/DOPD_historical_data.csv', 'TIDC': './WECC_compiled_historical_data/TIDC_historical_data.csv', 'CISO': './WECC_compiled_historical_data/CISO_historical_data.csv', 'PSCO': './WECC_compiled_historical_data/PSCO_historical_data.csv', 'PNM': './WECC_compiled_historical_data/PNM_historical_data.csv', 'SCL': './WECC_compiled_historical_data/SCL_historical_data.csv', 'IPCO': './WECC_compiled_historical_data/IPCO_historical_data.csv', 'IID': './WECC_compiled_historical_data/IID_historical_data.csv', 'BPAT': './WECC_compiled_historical_data/BPAT_historical_data.csv', 'WACM': './WECC_compiled_historical_data/WACM_historical_data.csv', 'PSEI': './WECC_compiled_historical_data/PSEI_historical_data.csv', 'PACW': './WECC_compiled_historical_data/PACW_historical_data.csv', 'PGE': './WECC_compiled

Local GCN

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import r2_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === GCN Model Definition ===
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, mlp_hidden_channels1, mlp_hidden_channels2, dropout_rate=0):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.fc1 = nn.Linear(hidden_channels, mlp_hidden_channels1)
        self.fc2 = nn.Linear(mlp_hidden_channels1, mlp_hidden_channels2)
        self.fc3 = nn.Linear(mlp_hidden_channels2, 1)
        self.dropout = nn.Dropout(p=dropout_rate)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x.squeeze()

results = []

for ba_index, ba_name in enumerate(ba_names):
    print(f"Training for BA: {ba_name}")

    #Prepare data for current BA
    data_list, val_data_list, test_data_list = [], [], []

    for t in range(x_tensor.shape[0]):
        y_ba = y_tensor[t][ba_index].unsqueeze(0)
        mask = torch.zeros(x_tensor.shape[1], dtype=torch.bool)
        mask[ba_index] = True
        data_list.append(Data(x=x_tensor[t], edge_index=edge_index, y=y_ba, ba_mask=mask))

    for t in range(x_val_tensor.shape[0]):
        y_val_ba = y_val_tensor[t][ba_index].unsqueeze(0)
        mask = torch.zeros(x_val_tensor.shape[1], dtype=torch.bool)
        mask[ba_index] = True
        val_data_list.append(Data(x=x_val_tensor[t], edge_index=edge_index, y=y_val_ba, ba_mask=mask))

    for t in range(x_test_tensor.shape[0]):
        y_test_ba = y_test_tensor[t][ba_index].unsqueeze(0)
        mask = torch.zeros(x_test_tensor.shape[1], dtype=torch.bool)
        mask[ba_index] = True
        test_data_list.append(Data(x=x_test_tensor[t], edge_index=edge_index, y=y_test_ba, ba_mask=mask))

    train_loader = DataLoader(data_list, batch_size=256, shuffle=True)
    val_loader = DataLoader(val_data_list, batch_size=256, shuffle=False)
    test_loader = DataLoader(test_data_list, batch_size=256, shuffle=False)

    #Model Setup
    model = GCN(in_channels=x_tensor.shape[2], hidden_channels=64,
                mlp_hidden_channels1=32, mlp_hidden_channels2=16).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    loss_fn = nn.MSELoss()

    best_val_loss = float('inf')
    patience = 10
    patience_counter = 0

    for epoch in range(150):
        model.train()
        total_loss = 0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            out = model(batch)
            pred = out[batch.ba_mask]
            loss = loss_fn(pred.view(-1), batch.y.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_train_loss = total_loss / len(train_loader)

        # === Validation for early stopping ===
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for val_batch in val_loader:
                val_batch = val_batch.to(device)
                out = model(val_batch)
                pred = out[val_batch.ba_mask]
                val_loss = loss_fn(pred.view(-1), val_batch.y.view(-1)).item()
                total_val_loss += val_loss
        avg_val_loss = total_val_loss / len(val_loader)
        scheduler.step(avg_val_loss)

        print(f"Epoch {epoch+1:02d} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    # === Final Validation ===
    model.load_state_dict(best_model_state)
    model.eval()
    y_pred, y_true = [], []

    with torch.no_grad():
        for test_batch in test_loader:
            test_batch = test_batch.to(device)
            out = model(test_batch)
            pred = out[test_batch.ba_mask]
            y_pred.extend(pred.cpu().view(-1).tolist())       
            y_true.extend(test_batch.y.cpu().view(-1).tolist())  

    # === Denormalize ===
    min_y = min_y_train_array[ba_index]
    max_y = max_y_train_array[ba_index]
    print(min_y, max_y)
    y_true_denorm = np.array(y_true) * (max_y - min_y) + min_y
    y_pred_denorm = np.array(y_pred) * (max_y - min_y) + min_y

    # === Compute Metrics ===
    # rms_abs = np.sqrt(mean_squared_error(y_true_denorm, y_pred_denorm))
    y_true_abs=np.mean(np.abs(y_true_denorm))
    rms_abs = np.sqrt(np.mean((y_pred_denorm - y_true_denorm) ** 2))
    rms_norm = rms_abs / np.mean(np.abs(y_true_denorm))
    mape = np.mean(np.abs(y_pred_denorm - y_true_denorm)) / np.mean(y_true_denorm)
    # mape = mean_absolute_percentage_error(y_true_denorm, y_pred_denorm)
    r2 = r2_score(y_true_denorm, y_pred_denorm)

    print(f" BA {ba_name} Validation Metrics:")
    print(f"  RMS_ABS:   {rms_abs:.4f}")
    print(f"  RMS_NORM:  {rms_norm:.4f}")
    print(f"  MAPE:      {mape:.4f}")
    print(f"  R²:        {r2:.4f}")
    print(f"  y_true_abs:        {y_true_abs:.4f}")

    # === Store ===
    results.append({
        "BA": ba_name,
        "RMS_ABS": rms_abs,
        "RMS_NORM": rms_norm,
        "MAPE": mape,
        "R2": r2,
        "y_true_abs": y_true_abs
    })

# === Save All Results ===
results_df = pd.DataFrame(results)
results_df.to_csv("validation_results_per_ba.csv", index=False)
print("All BA validation metrics saved to 'validation_results_per_ba.csv'")


Training for BA: AVA
Epoch 01 | Train Loss: 0.0168 | Val Loss: 0.0062
Epoch 02 | Train Loss: 0.0049 | Val Loss: 0.0051
Epoch 03 | Train Loss: 0.0034 | Val Loss: 0.0037
Epoch 04 | Train Loss: 0.0030 | Val Loss: 0.0035
Epoch 05 | Train Loss: 0.0025 | Val Loss: 0.0023
Epoch 06 | Train Loss: 0.0024 | Val Loss: 0.0023
Epoch 07 | Train Loss: 0.0021 | Val Loss: 0.0028
Epoch 08 | Train Loss: 0.0022 | Val Loss: 0.0020
Epoch 09 | Train Loss: 0.0021 | Val Loss: 0.0024
Epoch 10 | Train Loss: 0.0021 | Val Loss: 0.0019
Epoch 11 | Train Loss: 0.0021 | Val Loss: 0.0019
Epoch 12 | Train Loss: 0.0021 | Val Loss: 0.0022
Epoch 13 | Train Loss: 0.0020 | Val Loss: 0.0025
Epoch 14 | Train Loss: 0.0018 | Val Loss: 0.0024
Epoch 15 | Train Loss: 0.0019 | Val Loss: 0.0020
Epoch 16 | Train Loss: 0.0019 | Val Loss: 0.0019
Epoch 17 | Train Loss: 0.0017 | Val Loss: 0.0017
Epoch 18 | Train Loss: 0.0016 | Val Loss: 0.0016
Epoch 19 | Train Loss: 0.0016 | Val Loss: 0.0016
Epoch 20 | Train Loss: 0.0017 | Val Loss: 0.0016